In [1]:
import findspark, random
findspark.init("")

from pyspark import SparkContext, SparkConf
sc = SparkContext(appName="Assignment5", master="local[2]", conf=SparkConf().set('spark.ui.port', random.randrange(4000,5000)))

In [2]:
from spamminess import spamminess
from math import exp

def sequential_SGD(model, training_dataset='', delta = 0.002):
    #### Your solution to Question 1 here
    # open one of the training files - defaults to group_x
    with open(training_dataset) as f:
        #iterate through each line in the file
        for line in f:
            #separate the label and features and calculate the probability of spam or ham
            doc = line.split()
            t = doc[1]
            F = doc[2:]
            F = [int(feat) for feat in F]#convert features to integers
            score = spamminess(F,model)
            prob = 1.0/(1+exp(-score))
            #update the model weights based on the current documents label
            for feature in F:
                if t == 'spam':
                    if feature not in model:
                        model[feature] = (1.0-prob)*delta
                    else:
                        model[feature]+= (1.0-prob)*delta
                elif t=='ham':
                    if feature not in model:
                        model[feature] = (-prob)*delta
                    else:
                        model[feature]-= prob*delta
                    
    #   for line in f:
    #      each line represents a document
    #      read and parse the line
    #      Let:
    #        t represent the spam/ham tag for this document
    #        F represent the list of features for this document

    #      find the spamminess of the current document using the current model:
    #      score = spamminess(F,w)

    #      then, update the model:
    #      prob = 1.0/(1+exp(-score))
    #      for each feature f in F:
    #          if t == 'spam':
    #              increase model(f) by (1.0-prob)*delta (or set model(f) to (1.0-prob)*delta if f is not in the dict yet)
    #          elif t == 'ham':
    #              decrease model(f) by prob*delta (or set model(f) to -prob*delta if f is not in the dict yet)
    return model


In [3]:
# Your tests here
#Question 1
i=0
w = sequential_SGD({}) # Providing an empty model
for k,v in w.items():
    print(k,v)
    i+=1
    if i==50:
        break

387908 0.00695474119501872
697162 0.00695474119501872
426572 0.00695474119501872
161118 0.00695474119501872
688171 0.00695474119501872
43992 0.00695474119501872
908749 0.00695474119501872
126841 0.00695474119501872
116309 0.00695474119501872
950728 0.00695474119501872
394920 0.00695474119501872
358453 0.00695474119501872
23565 0.00695474119501872
210162 0.01394765848836145
861926 0.013942427517810815
177667 0.00695474119501872
284634 0.00695474119501872
358345 0.00695474119501872
971891 0.00695474119501872
646357 0.00695474119501872
599737 0.00695474119501872
957529 0.00695474119501872
970014 0.00695474119501872
449273 0.00695474119501872
129997 0.00695474119501872
244086 0.00695474119501872
529524 0.00695474119501872
170675 0.00695474119501872
126503 0.00695474119501872
455001 0.00695474119501872
648917 0.00695474119501872
255018 0.00695474119501872
716613 0.007877239977721872
106120 0.007877239977721872
310307 0.007877239977721872
318741 0.007877239977721872
169946 0.0015770186928456

#### Question 2 (5/20 marks)

Next, you should try implementing a Spark version of the SGD model trainer.   Before you start, create a new folder
called `models` inside your cs431 folder.   Your Spark implementation should read a training file, train the model, and then output the model to the models folder.  The model output file that you generate should list the weight associated with each feature, with one feature per line, like this:
```
(802123, 0.0009858585991850937)
(438450, 4.267897922108138e-05)
(271525, 0.0013133437007968654)
(92853, 0.0004300009932503611)
```

Use Spark's `saveAsTextFile` action to output your model.   For example, if you are training a model for the group_x training set, use `saveAsTextFile("models/group_x_model")`.   This will actually cause Spark to create a folder called `group_x_model`.   In the folder, there will be files with names like `part-00000` that contain the actual output data.  When you use `saveAsTextFile`, Spark will generate one `part-xxxxx` file for each partition of the RDD that you are writing out.   In this case, you should have only a single partition (for the reason described below), so there should be only one `part-xxxxx` file.

Training the SGD model is an inherently sequential task, since the training instances update the model one at a time, and each instance's spamminess is computed using the model produced by that instance's predecessors.   This means that the only part of the training that you can parallelize using Spark is the parsing of the input file.   Once the input is parsed, your Spark implementation will have to force all of the instances into a single partition, and then apply the training function to the entire partition.   To see whether you are getting sensible results, you can compare the model you learn with Spark to the one that you learned with your sequential Python program from Question 1.

Remember that training should occur entirely in Spark.  The training instances should never come into your driver program.

Implement the function `spark_SGD()` below that takes as input the path to the training dataset, an output path `output_model` and a value for the update parameter `delta`, and writes the trained model to `output_model` using Spark's `saveAsTextFile`. You can use it to generate models from all three of the training files, leaving the results in your models folder. For this assignment, you will be using Spark's original RDD interface, rather than the DataFrame interface.


In [4]:
from spamminess import spamminess
from math import exp
import shutil, os

def spark_SGD(training_dataset='', output_model='models/group_x_model', delta = 0.002):
    #helper function to calculate probability and update the model weights
    def update(x):
        model_Q2={}#dictionary to store the (features,weights) pairs
        for line in x:
            t = line[0]
            F = line[1]
            F = [int(feat) for feat in F]#convert feats to ints
            score = spamminess(F,model_Q2)
            prob = 1.0/(1+exp(-score))
            #update the model weights based on the current documents label
            for feature in F:
                if t == 'spam':
                    if feature not in model_Q2:
                        model_Q2[feature] = (1.0-prob)*delta
                    else:
                        model_Q2[feature]+= (1.0-prob)*delta
                elif t=='ham':
                    if feature not in model_Q2:
                        model_Q2[feature] = (-prob)*delta
                    else:
                        model_Q2[feature]-= prob*delta
        yield model_Q2#return a generator containing all (features,weights) pairs

    
    if os.path.isdir(output_model):
        shutil.rmtree(output_model) # Remove the previous model to create a new one
    training_data = sc.textFile(training_dataset)#read dataset
    labelFeature_rdd = training_data.map(lambda line:line.split()).map(lambda line:(line[1],line[2:])).coalesce(1)#obtain labels&features and combine all instances into 1 partition
    update_model = labelFeature_rdd.mapPartitions(update).flatMap(lambda line:[(int(k),v) for k,v in line.items()])#call function on whole partition
    update_model.saveAsTextFile(output_model)#save the model as text file
    #### Your Solution to Question 2 here



In [5]:
# Your tests here
spark_SGD()
spark_SGD(training_dataset='', output_model='models/group_y_model')
spark_SGD(training_dataset='', output_model='models/britney_model')

#### Question 3 (5/20 marks)

When you train a model using SGD, the model you get depends on the order in which you handle the training instances.  To see this in action, try using the Spark SGD trainer you implemented for Question 2 to train a model from the group_x training set, but with the instances processed in a different order.  

To do this, re-implement your trainer from Question 2 so that it will randomly reorder the training instances before using them to update the model. One way to shuffle the training instances is to assign a random sort key to each training instance as you read it from the input file, and then sort the instances using the random sort key.

Be sure that Spark is doing the work of shuffling the training instances.   Do not load the training instances into your driver program and sort them there.

Implement the function `spark_shuffled_SGD` below that takes as input the path to the training dataset, an output path `output_model` and a value for the update parameter `delta`, shuffles the training instances using the method described above and writes the trained model to `output_model` using Spark's `saveAsTextFile`.

Once you have implemented the shuffled trainer, train a model using shuffled group_x training instances, and compare the resulting model with group_x model you learned without shuffling.  It is up to you how to do this comparision.  At a minimum, compare features with the highest weights in each model to see if they are similar. You can also use the classifier in next question to classify documents using the two models, and compare results.


In [6]:
from spamminess import spamminess
from math import exp
import shutil, os, random

def spark_shuffled_SGD(training_dataset='', output_model='models/group_x_model', delta = 0.002):
    #helper function to calculate probability and update the model weights
    def update(x):
        model_Q2={}#dictionary to store the (features,weights) pairs
        for line in x:
            t = line[0]
            F = line[1]
            F = [int(feat) for feat in F]#convert feats to ints
            score = spamminess(F,model_Q2)
            prob = 1.0/(1+exp(-score))
            #update the model weights based on the current documents label
            for feature in F:
                if t == 'spam':
                    if feature not in model_Q2:
                        model_Q2[feature] = (1.0-prob)*delta
                    else:
                        model_Q2[feature]+= (1.0-prob)*delta
                elif t=='ham':
                    if feature not in model_Q2:
                        model_Q2[feature] = (-prob)*delta
                    else:
                        model_Q2[feature]-= prob*delta
        yield model_Q2#return a generator containing all (features,weights) pairs
    
    
    
    if os.path.isdir(output_model):
        shutil.rmtree(output_model) # Remove the previous model to create a new one
    training_data = sc.textFile(training_dataset)#read dataset
    total_data = training_data.count()
    #assign random keys to each data point, sort it by the keys which shuffles the data and then repartition into 1 single partition
    labelFeature_rdd = training_data.map(lambda line:line.split())\
                        .map(lambda line:(random.randint(0,total_data),(line[1],line[2:])))\
                        .sortByKey().map(lambda line:(line[1][0],line[1][1])).coalesce(1)
    update_model = labelFeature_rdd.mapPartitions(update).flatMap(lambda line:[(int(k),v) for k,v in line.items()])#call function on whole partition
    update_model.saveAsTextFile(output_model)#save the model as text file
    #### Your Solution to Question 3 here




In [7]:
# Your tests here
spark_shuffled_SGD(output_model='models/group_x_model_shuffled')


#### Question 4 (5/20  marks)

Last but not least, you should write a Spark program that can be used to classify documents as spam or ham, using the classification models you produced.

The test data, i.e., the document instances that you should classifiy, are located in the file
* ``

Each line in this file represents a document that needs to be classified as spam or ham.  The format of this file is identical to the format of the files that hold the training instances.

Implement the function `spark_classify` below that will load a model (from a specified folder under `models`), classify all of the instances in a given test data file (`spam.test.qrels.txt` by default) using that model, and then output the results in the folder `results_path` using Spark's `saveAsTextFile`.   The contents of the output file should look like this:
```
(clueweb09-en0000-00-00142,spam,2.601624279252943,spam)
(clueweb09-en0000-00-01005,ham,2.5654162439491004,spam)
(clueweb09-en0000-00-01382,ham,2.5893946346394188,spam)
```
Each line of the output represents one test instance.   The first two fields are the document ID and the test label.  These are just copied from the test data.   The third field is the spamminess score of the document, produced by the spamminess function using the model you are classifying with.   The fourth field is the spam/ham prediction made by the model.

Of course, your spam/ham classifier must **not** use the test label from the input when making its prediction.  The test labels are the "ground truth" against which your predictions are being compared.   Using them to make predictions would defeat the whole purpose of model-based classification.

Make sure that classification of the test instances is done by Spark, not by your driver program.  Do ***not*** load the test instances or classification results into your driver program. You are however allowed to load the model weights into your driver program to distribute them as side data. 
Unlike model training, classification is easily parallelizable, since each document is classified independently. 

In [14]:
from spamminess import spamminess
from math import exp
import shutil, os

def spark_classify(input_model='models/britney_model', test_dataset='', results_path='results/test_qrels/model_britney_predictions'):
    
    
    if os.path.isdir(results_path):
        shutil.rmtree(results_path) # Remove the previous results
    test_data = sc.textFile(test_dataset)#read dataset
    labelFeature_rdd = test_data.map(lambda line:line.split()).map(lambda line:(line[0],line[1],[int(i) for i in line[2:]]))#split lines into docID,label and features
    curr_model = sc.textFile(input_model).map(eval).collectAsMap()#store model as dictionary
    spam_scores = labelFeature_rdd.map(lambda line:(line[0],line[1],spamminess(line[2],curr_model)))#calculate spamminess score
    spam_predictions = spam_scores.map(lambda line:(line[0],line[1],line[2],'spam' if (line[2])>=0 else 'ham'))#classify each doc based on spamminess score
    spam_predictions.saveAsTextFile(results_path)#save predictions
    #### Your Solution to Question 4 here

In [15]:
spark_classify()

## Comparison for Q3

- The cell below runs the program to compute the ROC value or more specifically (1-ROCA)% value for a given models predictions
- As we can see below the results are different for "Model X" and "Model X Shuffled" even though its the same training file but I have only changed the order in which the training instances are read
- This proves that shuffling around the data does effect the model weights and inturn effects the ROCA score i.e the performance of the classifier